# Modeling government voting systems

The code below is used to model first-past-the-post (= plurality) voting systems (for now; I may add other systems later). I was interested how many votes are typically 'dropped' in such a voting system.

The code can then be used to run a number of elections (e.g. 100,000) and see the average number of votes dropped overall.

Assumptions:

- the votes per party are drawn at random from a gaussian distribution of 1,000 numbers (rounded to nearest 4 decimals). Gaussianity is a reasonable assumption (see e.g. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3478593/)

- alternatively (if there's no mean and standard deviation) votes per party are drawn simply by random choice of integers from 0-100. This assumption is extremely naive and not recommended


Inputs:

- a list of the number of constituents per MP (can be generic, e.g. 10,000 constituents per MP, or even more specific if data on constituents per MP are available)

- number of parties: int

- number of elections: int

- mean: float (mean of percentage of votes cast for all parties), default=None

- stdev: float (standard deviation of votes cast for all parties), default=None

- party parameters: list of tuples where each tuple's first element is the mean votes of the respective party (float) and the second element is that same party's standard deviation (float), default=None

- verbose: Boolean

Outputs:

- percentage of votes dropped

- number of votes dropped

- if verbose==True, further outputs are: an np.array with percentages of votes, dictionary of constituents per party

In [119]:
import numpy as np
from collections import Counter
import random

In [372]:
def simple_constituency_election(n_parties, n_constituents, verbose=False):
    n = n_parties
    percents = np.arange(1,101)
    i = 0
    percent_lst = []
    
    while n-i > 1:
        try:
            percent_votes = np.random.choice(percents)
            percents = np.flip(np.delete(np.flip(percents), np.arange(percent_votes)))
            percent_lst.append(percent_votes)
            i += 1
        except:
            i += 1
    
    # appending the last party's percentage to the list and finding the max (i.e. which party had the highest % of votes)
    # the exception is necessary here, if array empty the .max() will return an error
    try:
        percent_lst.append(percents.max())
    except:
        percent_lst.append(0)
    
    if len(percent_lst) <= n:
        diff = n - len(percent_lst)
        percent_lst += [0] * diff
    
    percent_dropped = 100 - max(percent_lst)
    percent_arr = np.array(percent_lst)
    
    if verbose:
        vote_distribution = {}
        for ix,ele in enumerate(percent_lst):
            vote_distribution['party_' + str(ix+1)] = n_constituents * ele/100

        return percent_dropped, percent_arr, vote_distribution
    
    return percent_dropped

In [373]:
# unit test
simple_constituency_election(3, 100, verbose=True)

(19, array([81, 19,  0]), {'party_1': 81.0, 'party_2': 19.0, 'party_3': 0.0})

In [316]:
def constituency_election_gaussian(n_parties, n_constituents, mean, stdev, verbose=False):
    n = n_parties
    percents = np.arange(0,100, 0.0001) # these are the percentage points possible
    i = 0
    percent_lst = []
    vote = -1 # a negative number that is not going to be in the percents variable (any neg. number or any extremely large number will do)
    
    while n-i > 1:

        while vote not in percents:
            normal = np.random.normal(mean, stdev, 1000)
            normal[normal<0] = 0
            vote = np.round(np.random.choice(normal), 4)
#             vote = np.round(np.random.choice(np.abs(np.random.normal(mean, stdev, 1000))), 4)

        percents = np.flip(np.delete(np.flip(percents), np.arange(len(np.arange(0,vote, 0.0001))))) # np.arange(len(np.arange(0,vote, 0.0001))): the first arange is to get the length of the array (in 0.0001 increments) until the vote. the second arange then is to generate a number of integers, which will be the indices to drop by np.delete
        percent_lst.append(vote)
        i += 1
        vote = -1
    
    # appending the last party's percentage to the list and finding the max (i.e. which party had the highest % of votes)
    # the exception is necessary here, if array empty the .max() will return an error
    try:
        percent_lst.append(np.round(percents.max(), 4))
    except:
        percent_lst.append(0)
        
    percent_dropped = 100 - max(percent_lst)
    percent_arr = np.array(percent_lst)
    
    if verbose:
        vote_distribution = {}
        for ix,ele in enumerate(percent_lst):
            vote_distribution['party_' + str(ix+1)] = n_constituents * ele/100

        return percent_dropped, percent_arr, vote_distribution
    
    return percent_dropped

In [317]:
# unit test and sanity check
d = np.array([15.98, 6.55, 1.62, 33.12, 34.34, 7.63]) # election outcome of last canadian election
d_mu = d.mean()
d_s = d.std()
i, arr, dct = constituency_election_gaussian(n_parties=5, n_constituents=1500, mean=d_mu, stdev=d_s, verbose=True)

In [318]:
sum(v for v in dct.values())

1499.9985

In [319]:
arr.sum()

99.9999

In [332]:
def party_specific_gaussian_election(n_parties, n_constituents, party_parameters, verbose=False):
    n = n_parties
    assert(len(party_parameters) == n)

    
    def one_run(shuffled, percents, percent_dct, party_parameters, switch=False):
        vote = -1 # a negative number that is not going to be in the percents variable (any neg. number or any extremely large number will do)
        for i in shuffled:
            party_mean = party_parameters[i][0]
            party_stdev = party_parameters[i][1]
            
            if switch == False:
                count = 0
                while vote not in percents:
                    count += 1
                    
                    if count >= 20: # this is to stop the code from running forever
                        scaled_mean = np.round(percents.max()/100 * party_mean, 4)
                        scaled_stdev = np.round(percents.max()/100 * party_stdev, 4)
                        normal = np.random.normal(scaled_mean, scaled_stdev, 1000)
                        normal[normal<0] = 0
                        vote = np.round(np.random.choice(normal),4)
#                         vote = np.round(np.random.choice(np.abs(np.random.normal(scaled_mean, scaled_stdev, 1000))), 4) # the problem with this is that it takes the absolute value of the gaussian and if the mean is close to 0 (= small party) then that particular party will do better than the mean that was input for it as a parameter

                    elif count >= 50:
                        scaled_mean = np.round(percents.max()/100 * party_mean, 4)
                        scaled_stdev = np.round(percents.max()/100 * party_stdev, 4)
                        normal = np.random.normal(scaled_mean/count, scaled_stdev/count, 1000)
                        normal[normal<0] = 0
                        vote = np.round(np.random.choice(normal),4)
#                         vote = np.round(np.random.choice(np.abs(np.random.normal(scaled_mean/count, scaled_stdev/count, 1000))), 4)

                    else:
                        normal = np.random.normal(party_mean, party_stdev, 1000)
                        normal[normal<0] = 0
                        vote = np.round(np.random.choice(normal),4)
#                         vote = np.round(np.random.choice(np.abs(np.random.normal(party_mean, party_stdev, 1000))), 4)
            else: 
                # the problem with this method is that percents.max() will be smaller for each party further down the party_parameters list, hence the 10 iterations below
                scaled_mean = np.round(percents.max()/100 * party_mean, 4)
                scaled_stdev = np.round(percents.max()/100 * party_stdev, 4)
                count = 0
                while vote not in percents:
                    count += 1
                    if count >= 30:
                        normal = np.random.normal(scaled_mean/count, scaled_stdev/count, 1000)
                        normal[normal<0] = 0
                        vote = np.round(np.random.choice(normal),4)
#                         vote = np.round(np.random.choice(np.abs(np.random.normal(scaled_mean/count, scaled_stdev/count, 1000))), 4)

                    else:
                        normal = np.random.normal(scaled_mean, scaled_stdev, 1000)
                        normal[normal<0] = 0
                        vote = np.round(np.random.choice(normal),4)
#                         vote = np.round(np.random.choice(np.abs(np.random.normal(scaled_mean, scaled_stdev, 1000))), 4)
            
            percents = np.flip(np.delete(np.flip(percents), np.arange(len(np.arange(0,vote, 0.0001))))) # np.arange(len(np.arange(0,vote, 0.0001))): the first arange is to get the length of the array (in 0.0001 increments) until the vote. the second arange then is to generate a number of integers, which will be the indices to drop by np.delete
            percent_dct[i].append(vote)
            vote = -1
        return percents, percent_dct

###### this code block iterates the above 10 times, then takes avg of result
    for j in range(10):

        percent_dct = {k: [] for k in range(n)}
        percents = np.arange(0,100, 0.0001) # these are the percentage points possible
        shuffled = np.arange(n)
        np.random.shuffle(shuffled) # shuffles in-place
        
        percents, percent_dct = one_run(shuffled, percents, percent_dct, party_parameters, False) # this is to speedup the code

        while percents.max() > 1: # if there is more than one percentage point remaining, the loop runs again
            percents, percent_dct = one_run(shuffled, percents, percent_dct, party_parameters, True)

        for i in range(n):
            percent_dct[i].append(percents.max()/n) # the remaining of the unallocated percentage decimals are then equally distributed over the party percentages

            if j == 0:
                percent_arr = np.array([sum(lst) for lst in percent_dct.values()])
            else:
                tmp_percent_arr = np.array([sum(lst) for lst in percent_dct.values()])
                percent_arr = np.vstack((percent_arr, tmp_percent_arr))
    
    percent_arr = np.mean(percent_arr, axis=0)
    
    # since we've taken the average of 10 runs, the percentages might not add up to 100
    # hence we sum the array and if different from 100, we add difference to each column (=party) equally
    diff = 100 - percent_arr.sum()
    
    if diff != 0:
        percent_arr += (np.ones(n) * diff/3)
        
    np.allclose(100, percent_arr.sum())
######

    percent_dropped = 100 - percent_arr.max()
    
    if verbose:
        vote_distribution = {}
        for i in range(percent_arr.shape[0]):
            vote_distribution['party_' + str(i+1)] = n_constituents * percent_arr[i]/100

        return percent_dropped, percent_arr, vote_distribution
    
    return percent_dropped

In [333]:
%%time
# unit test
# not perfect but hopefully a good enough heuristic
party_parameters = [(42, 4), (2, 1), (56, 0.1)]

for i in range(20):
#     print(i)
    dropped, arr, dist = party_specific_gaussian_election(n_parties=3, n_constituents=100, party_parameters=party_parameters, verbose=True)
    if i == 0:
        stacked = arr
    else:
        stacked = np.vstack((stacked, arr))
print(dropped, arr, dist)
print('mean', np.mean(stacked, axis=0))
print('std', np.std(stacked, axis=0))

44.31665833333333 [42.34376667  1.97289167 55.68334167] {'party_1': 42.34376666666667, 'party_2': 1.9728916666666647, 'party_3': 55.68334166666668}
mean [43.80962714  2.12238488 54.06798798]
std [1.60665134 0.35280108 1.69733626]
Wall time: 10.6 s


In [375]:
def general_election(n_parties, n_constituencies, iterations=1, mean=None, stdev=None, party_parameters=None,verbose=False):
    percent_dropped = 0
    percent_arr = np.zeros(n_parties)
    vote_distribution = Counter()
    
    for iteration in range(iterations):
        print('iteration: ', iteration)
        
        for n_constituents in n_constituencies:
            
            if mean == None and party_parameters == None:
                if not verbose:
                    percent_dropped += simple_constituency_election(n_parties, n_constituents, verbose=verbose)
                else:
                    tmp_percent_dropped,tmp_percent_arr,tmp_vote_distribution = simple_constituency_election(n_parties, n_constituents, verbose=verbose)
                    percent_dropped += tmp_percent_dropped
                    percent_arr += tmp_percent_arr
                    vote_distribution += tmp_vote_distribution
            
            elif party_parameters != None:
                if not verbose:
                    percent_dropped += party_specific_gaussian_election(n_parties, n_constituents, party_parameters, verbose=verbose)
                else:
                    tmp_percent_dropped,tmp_percent_arr,tmp_vote_distribution = party_specific_gaussian_election(n_parties, n_constituents, party_parameters, verbose=verbose)
                    percent_dropped += tmp_percent_dropped
                    percent_arr += tmp_percent_arr
                    vote_distribution += tmp_vote_distribution
            else: 
                if not verbose:
                    percent_dropped += constituency_election_gaussian(n_parties, n_constituents, mean, stdev, verbose=verbose)
                else:
                    tmp_percent_dropped,tmp_percent_arr,tmp_vote_distribution = constituency_election_gaussian(n_parties, n_constituents, mean, stdev, verbose=verbose)
                    percent_dropped += tmp_percent_dropped
                    percent_arr += tmp_percent_arr
                    vote_distribution += (tmp_vote_distribution)

    percent_dropped = percent_dropped/(len(n_constituencies) * iterations)

    if verbose:
        return percent_dropped, percent_arr/len(n_constituencies * iterations), {k: round(v/iterations) for k,v in vote_distribution.items()}
    else:
        return percent_dropped
    

In [379]:
dropped, arr, dct = general_election(n_parties=6, n_constituencies=[10000] * 300, iterations=100, mean=None, stdev=None, party_parameters=None,verbose=True)
dropped

iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  11
iteration:  12
iteration:  13
iteration:  14
iteration:  15
iteration:  16
iteration:  17
iteration:  18
iteration:  19
iteration:  20
iteration:  21
iteration:  22
iteration:  23
iteration:  24
iteration:  25
iteration:  26
iteration:  27
iteration:  28
iteration:  29
iteration:  30
iteration:  31
iteration:  32
iteration:  33
iteration:  34
iteration:  35
iteration:  36
iteration:  37
iteration:  38
iteration:  39
iteration:  40
iteration:  41
iteration:  42
iteration:  43
iteration:  44
iteration:  45
iteration:  46
iteration:  47
iteration:  48
iteration:  49
iteration:  50
iteration:  51
iteration:  52
iteration:  53
iteration:  54
iteration:  55
iteration:  56
iteration:  57
iteration:  58
iteration:  59
iteration:  60
iteration:  61
iteration:  62
iteration:  63
iteration:  64
iteration:  65
iteration:  66
itera

37.3102

In [335]:
%%time
# unit test and sanity check
n_constituencies = [10000] * 300
n_parties = 6
d = np.array([15.98, 6.55, 1.62, 33.12, 34.34, 7.63]) # election outcome of last canadian election
d_mu = d.mean()
d_s = d.std()
dropped, arr, dct = general_election(n_parties=n_parties, n_constituencies=n_constituencies, iterations=2, mean=d_mu, stdev=d_s, verbose=True)
# print(dropped)

iteration:  0
iteration:  1
Wall time: 32.1 s


In [336]:
# assert (sum(n_constituencies) == sum(dct.values()))
# sum(dct.values())
dropped

61.89028166666671

In [337]:
party_parameters = [(15.98,5), (6.55,2), (1.62, 0.4), (33.12,7), (34.34,8), (7.63,2)]
dropped, arr, dct = general_election(n_parties=6, n_constituencies=n_constituencies, iterations=2, party_parameters=party_parameters, verbose=True)

iteration:  0
iteration:  1


In [338]:
dropped

63.89132409848475

In [339]:
arr

array([15.83361004,  6.59954513,  1.78915787, 33.40174202, 34.91542923,
        7.68692156])